In [1]:
import pandas as pd
import sqlalchemy as db

In [2]:
engine = db.create_engine("mssql+pyodbc://@cloudcds_ts",echo=False)
connection = engine.connect()
metadata = db.MetaData(engine,schema='tgl_hsio')

# Data Pull

## CDS Database Connection and HSIO Tables: joinID_view

In [3]:
all_t = db.Table('batched_execution_joinID_view', metadata, autoload=True, autoload_with=engine)

In [4]:
from datetime import datetime, timedelta
time_hours_back = datetime.now() - timedelta(hours=72) #pull data in last 12 hour 
#q = db.select([all_t]).where(db.and_(all_t.columns.inittime >= time_hours_back,all_t.columns.InnerCaseID == 'usb0_g2'))
q = db.select([all_t]).where(all_t.columns.inittime >= time_hours_back)
result = connection.execute(q).fetchall()
df = pd.DataFrame(result)
df.columns = result[0].keys()
df = df.rename(columns={'testid': 'TestId'}).copy()
df

,TestId,inittime,username,hostname,JoinID,MainCaseID,InnerCaseID,Repetition,Iteration,DataValid
0,53646374-B906-4D1F-90DB-1033785AC64F,2021-03-09 22:10:31.2393472,AMR\jfsvuser,JF04WVAW0397,1,c0,None,1,NaN,1
1,53646374-B906-4D1F-90DB-1033785AC64F,2021-03-09 22:10:31.2393472,AMR\jfsvuser,JF04WVAW0397,2,c0,DpoC_1p62_all_no_ssc_keysight,1,NaN,1
2,53646374-B906-4D1F-90DB-1033785AC64F,2021-03-09 22:10:31.2393472,AMR\jfsvuser,JF04WVAW0397,3,c0,DpoC_1p62_all_no_ssc_keysight,1,1.0,1
3,53646374-B906-4D1F-90DB-1033785AC64F,2021-03-09 22:10:31.2393472,AMR\jfsvuser,JF04WVAW0397,4,c0,DpoC_2p7_all_no_ssc_keysight,1,NaN,1
4,53646374-B906-4D1F-90DB-1033785AC64F,2021-03-09 22:10:31.2393472,AMR\jfsvuser,JF04WVAW0397,5,c0,DpoC_2p7_all_no_ssc_keysight,1,1.0,1
...,...,...,...,...,...,...,...,...,...,...
7552,321EA76F-02F3-4743-A882-D2FFDFCE0675,2021-03-10 12:21:34.6769703,AMR\jfsvuser,JF04WVAW0362,8,c4-lqtx3,peglane0m,3,NaN,1
7553,321EA76F-02F3-4743-A882-D2FFDFCE0675,2021-03-10 12:21:34.6769703,AMR\jfsvuser,JF04WVAW0362,9,c4-lqtx3,peglane0m,3,1.0,1
7554,321EA76F-02F3-4743-A882-D2FFDFCE0675,2021-03-10 12:21:34.6769703,AMR\jfsvuser,JF04WVAW0362,10,c4-lqtx3,None,4,NaN,1
7555,321EA76F-02F3-4743-A882-D2FFDFCE0675,2021-03-10 12:21:34.6769703,AMR\jfsvuser,JF04WVAW0362,11,c4-lqtx3,peglane0m,4,NaN,1


###### List of unique TestIds from JoinID_view

In [ ]:
testIDs = list(df['TestId'].drop_duplicates())
testIDs

###### Query PerLaneVertical Data

In [ ]:
plv_t = db.Table('radphy_PerLaneVertical', metadata, autoload=True, autoload_with=engine)
q = db.select([plv_t]).where(plv_t.columns.TestId.in_(testIDs))
result = connection.execute(q).fetchall()
df_plv = pd.DataFrame(result)
df_plv.columns = result[0].keys()
df_plv

###### Drop "RowId" columns

In [ ]:
df_plv = df_plv.drop(columns="RowId")
df_plv

## Get Per Lane Veritical Table where Split By( :Key )

In [ ]:
df_plv_split = df_plv.pivot_table(index=['TestId', 'SubTestId', 'JoinID', 'Interface','Port', 'Lane', 'Type', 'BlockName'],columns="Key",values="Value",aggfunc=lambda x: ' '.join(x))
df_plv_split = df_plv_split.reset_index()
df_plv_split

## MetaData pull

In [ ]:
meta_t = db.Table('radphy_metadata', metadata, autoload=True, autoload_with=engine)

In [ ]:
q = db.select([meta_t]).where(meta_t.columns.TestId.in_(testIDs))
result = connection.execute(q).fetchall()
df_meta = pd.DataFrame(result)
df_meta.columns = result[0].keys()
df_meta

###### list of unique Testids from metadata 

In [ ]:
metadata_testids = list(df_meta['TestId'].drop_duplicates())
metadata_testids

## MetaData split where Split By( :PortValueName )

In [ ]:
metadata_split = df_meta.pivot_table(index=['TestId', 'SubTestId', 'ExperimentType', 'TestName','Description', 'Comment', 'Interface','Port'],columns='PortValueName',values='PortValue',aggfunc=lambda x: ' '.join(x))
metadata_split = metadata_split.reset_index()
metadata_split

## DfxSummary pull

In [ ]:
dfx_summary_t = db.Table('radphy_DfxSummary', metadata, autoload=True, autoload_with=engine)

In [ ]:
q = db.select([dfx_summary_t]).where(dfx_summary_t.columns.TestId.in_(testIDs))
result = connection.execute(q).fetchall()
df_dfx_summary = pd.DataFrame(result)
df_dfx_summary.columns = result[0].keys()
df_dfx_summary

## Vertical Table for DfxSummary where Split By( :DfxKey, :DfxType, :DfxDirection ))

In [ ]:
df_dfx_summary_split = df_dfx_summary.pivot_table(index=['TestId', 'SubTestId', 'JoinID', 'Interface','Port', 'Lane'],columns=['DfxType','DfxDirection','DfxKey'],values="DfxVal",aggfunc=lambda x: ' '.join(x))
df_dfx_summary_split = df_dfx_summary_split.reset_index()
df_dfx_summary_split

## DfxDetails Pull

In [ ]:
dfx_details_t = db.Table('radphy_DfxDetails', metadata, autoload=True, autoload_with=engine)

In [ ]:
q = db.select([dfx_details_t]).where(dfx_details_t.columns.TestId.in_(testIDs))
result = connection.execute(q).fetchall()
dfx_details_t = pd.DataFrame(result)
dfx_details_t.columns = result[0].keys()
dfx_details_t

## DfxDetails Split where Split By( :DfxKey, :DfxType, :DfxDirection ))

In [ ]:
df_dfx_details_split = dfx_details_t.pivot_table(index=['TestId', 'SubTestId', 'JoinID', 'Interface','Port', 'Lane'],columns=['DfxType','DfxDirection','DfxKey'],values="DfxVal",aggfunc=lambda x: ' '.join(x))
df_dfx_details_split = df_dfx_details_split.reset_index()
df_dfx_details_split

## SutInfo Pull

In [ ]:
sut_info_t = db.Table('radphy_sut_info', metadata, autoload=True, autoload_with=engine)

In [ ]:
q = db.select([sut_info_t]).where(sut_info_t.columns.TestId.in_(testIDs))
result = connection.execute(q).fetchall()
sut_info_t = pd.DataFrame(result)
sut_info_t.columns = result[0].keys()
sut_info_t

## PerLanePerFreqVertical

In [ ]:
plvFreqVertical_t = db.Table('radphy_PerLanePerFreqVertical', metadata, autoload=True, autoload_with=engine)

In [ ]:
q = db.select([plvFreqVertical_t]).where(plvFreqVertical_t.columns.TestId.in_(testIDs))
result = connection.execute(q).fetchall()
if len(result) > 0:
    plvFreqVertical_t.columns = result[0].keys()
    plvFreqVertical_t = pd.DataFrame(result)
    plvFreqVertical_t
    plvFreqVertical_split = plvFreqVertical_t.pivot_table(index=['TestId', 'SubTestId', 'JoinID', 'Interface','Port', 'Lane', 'Frequency'], columns='Key', values='Value',aggfunc=lambda x: ' '.join(x))
    plvFreqVertical_split = plvFreqVertical_split.reset_index()
    plvFreqVertical_split
else:
    plvFreqVertical_split = False

## Baseline Table Formation 

##### Join PerLaneVertical with joinSqlStatement

In [ ]:
#df = joinID_view tbale
#df_plv_split = split table for PerLane Vertical
df.to_csv("c:\\intel\\hsio_or_data\\df.csv")
df_plv_split.to_csv("c:\\intel\\hsio_or_data\\plv_split.csv")
jplv = pd.merge(df,df_plv_split,on = 'TestId')
baseline = jplv
baseline
jplv.to_csv("c:\\intel\\hsio_or_data\\baseline.csv")

###### Join Dfx summary split with baseline

In [ ]:
DfxJoin = pd.concat([baseline,df_dfx_summary_split],axis = 1)
baseline = DfxJoin
DfxJoin

###### Join Dfx details split with baseline

In [ ]:
DfxDetailJoin = pd.concat([baseline,df_dfx_details_split],axis = 1)
baseline = DfxDetailJoin
DfxDetailJoin

###### Join Baseline with plvFreqVertical_split(If it exists)

In [ ]:
if plvFreqVertical_split != False:
    freqvertical = pd.concat([baseline,plvFreqVertical_split],axis = 1)
    baseline = freqvertical
    freqvertical

###### Join Baseline with metadata split

In [ ]:
metaJoin = pd.concat([baseline,metadata_split],axis = 1)
baseline = metaJoin
metaJoin

###### Join Baseline with sutinfo table 

In [ ]:
sutJoin = pd.concat([baseline,sut_info_t],axis = 1)
baseline = sutJoin
sutJoin

###### save data

In [ ]:
import os
now = datetime.now()
_str = now.strftime("%d_%m_%Y_%H_%M_%S")
if not os.path.isdir("c:\\intel\\hsio_or_data"):
    print("Creating directory: c:\\intel\\hsio_or_data")
    os.mkdir("c:\\intel\\hsio_or_data")
_path = "c:\\intel\\hsio_or_data"
save_file_path = os.path.join(_path, "jupyter_pull_data_{}.csv".format(_str))
save_file_path

In [ ]:
baseline.to_csv(save_file_path)

###### Debug window:

In [ ]:
f_plv_split.to_csv("c:\\intel\\hsio_or_data\\baseline.csv")